In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

import scipy.io
from fastbook import *
from fastai.vision.widgets import *
from fastai.text.all import *
from fastai import * 
from fastai.text import * 

In [ ]:
path     = untar_data('https://s3.amazonaws.com/fast-ai-nlp/yelp_review_full_csv.tgz')
Path.BASE_PATH = path

I will do this homework with 1000 datapoints as otherwise, even with GPU, it does not work

In [ ]:
data = pd.read_csv((path/'train.csv'), names = ['label', 'text'])
data = data.drop(list(range(1,649001)))
data.shape

(1000, 2)

Loading the data into batches of size 65 with length of sequencce 72

In [ ]:
imdb_lm = DataBlock(blocks   = TextBlock.from_df('text', is_lm=True),
                    get_x    = ColReader('text'),
                    splitter = RandomSplitter(0.1))

dls = imdb_lm.dataloaders(data, bs=64, seq_len=72)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Here we can see all the features which are automatically handled by fastai like tokenization. This is relevant for the algorithm as it ensures an efficient differentiation between the words and therefore allows to weigh certain words more than others

In [ ]:
dls.show_batch(max_n=2)

,text,text_
0,xxbos xxmaj nice .. casual .. great service ( vegas great ) .. \n xxmaj east access .. sweet setting .. cool people . xxunk music .. and off course .. great looking service xxbos xxmaj this is cool place to eat . xxmaj great atmosphere and good food . xxmaj good people . xxmaj food has been great every time i go . xxbos xxmaj xxunk made a xxup great xxup,xxmaj nice .. casual .. great service ( vegas great ) .. \n xxmaj east access .. sweet setting .. cool people . xxunk music .. and off course .. great looking service xxbos xxmaj this is cool place to eat . xxmaj great atmosphere and good food . xxmaj good people . xxmaj food has been great every time i go . xxbos xxmaj xxunk made a xxup great xxup cocktail
1,"her friend 's birthday party . xxmaj it worked out very well . xxmaj the group picked out the pieces they wanted to do , xxunk and chatted away for about an hour , cleaned up together , and then finished with gifts and a xxunk of "" happy xxmaj birthday to xxmaj you "" . xxmaj it was superb to have kids and parents playing + working together for an activity","friend 's birthday party . xxmaj it worked out very well . xxmaj the group picked out the pieces they wanted to do , xxunk and chatted away for about an hour , cleaned up together , and then finished with gifts and a xxunk of "" happy xxmaj birthday to xxmaj you "" . xxmaj it was superb to have kids and parents playing + working together for an activity for"


Building the model as in the book

In [ ]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.3, metrics=[accuracy, Perplexity()]).to_fp16()

fit a few epochs

In [ ]:
learn.fit_one_cycle(5, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.734847,4.590147,0.226531,98.508865,00:24
1,4.515180,4.328839,0.246579,75.856186,00:23
2,4.328253,4.218273,0.251687,67.916107,00:23
3,4.191616,4.174661,0.252590,65.017807,00:23
4,4.114514,4.167120,0.252284,64.529305,00:23


In [ ]:
learn.save_encoder('fine_tuned')

Here are the generations of text. I used the word "moscow" just for fun. However as those are reviews from america about restaurants, it probablu does not reallt now wgat to to with "moscow". However, the word "moscow" must have been among the 1000 messages I used, as the
 algorithm is able to recocnize it!


In [ ]:
[learn.predict("The people in Moscow", n_words = 10, temperature=0.75) for _ in range(5)]

['The people in Moscow are women , and Vietnamese women have been involved',
 'The people in Moscow together , including Steve Wynn and Billy',
 'The people in Moscow have a similar response to the First People',
 'The people in Moscow are Japanese - speaking people , who are',
 'The people in Moscow during the Cold " Day "']

This is somehow more logical, but still not good. However, more using more than 1000 should resolve this problem.

In [ ]:
[learn.predict("The restaurant", n_words = 15, temperature=0.75) for _ in range(5)]

['The restaurant is open to the public on weekends and is open on Friday .',
 'The restaurant in Las Vegas , Las Vegas , is known as the',
 'The restaurant that Expensive Taste was supposed to host was the House of',
 'The restaurant was opened with a restaurant on the same day as The Restaurant .',
 'The restaurant in Dallas , Texas is located close to the Dallas Wings']

Now I build the data block for the classification problem with the necessary vocab

In [ ]:
imdb_clas = DataBlock(blocks=(TextBlock.from_df('text', seq_len=72, vocab=dls.vocab), CategoryBlock),
                      get_x    = ColReader('text'),
                      get_y    = ColReader('label'),
                      splitter = RandomSplitter(0.1))

dls = imdb_clas.dataloaders(data, bs=64)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
learn_new = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy).to_fp16()

In [ ]:
learn_new = learn_new.load_encoder('fine_tuned')

And the results are not very good. haha. But one should be able to improve this with more observations as I said.

In [ ]:
learn_new.show_results(max_n=2)

,text,category,category_
0,"xxbos xxmaj why ? xxmaj why ? xxmaj why … can we not get more solid xxup bbq here ? xxmaj has xxmaj vegas been xxunk by some sort of barbecue xxunk xxunk ? xxmaj have the xxup bbq xxmaj xxunk no xxunk on xxmaj sin xxmaj city ? i really just do n't get it . xxmaj but before i self - xxunk , let me explain … \n\n xxmaj bubba xxmaj d 's is a brand new xxup bbq joint located inside the xxmaj town xxmaj center xxmaj lounge on xxmaj xxunk xxmaj xxunk , just off xxmaj xxunk & the xxunk . xxmaj i 'm not super familiar with this lounge , but i do know from driving by it everyday that the restaurant inside has seen its fair share of changes . xxmaj not long ago , a breakfast place used to be here that i",2,1
1,"xxbos xxmaj first , let me say , i am so happy to have xxmaj culinary xxmaj dropout open in xxmaj tempe in the xxmaj farmer xxmaj xxunk xxmaj district . i think it is going to be an xxunk establishment for lots of other development in this area and talk about a well run establishment , wow , one can almost hear the xxunk of xxunk efficiency in how this place is run and it is a big space and so popular ! i took my work - team to lunch here today and honestly , no detail is xxunk and i have no doubt that in going here folks will have a consistently positive experience . \n\n xxmaj as for why it is only marked as a - ok right now - well , my first time at this location and as with the other culinary dropout i",3,1


In [ ]:
interp = ClassificationInterpretation.from_learner(learn_new)

In [ ]:
interp.plot_top_losses(k=2)

,input,target,predicted,probability,loss
0,"xxbos xxmaj yelp xxunk you to xxunk xxup characters in your reviews -- which simply is not enough for things like "" experiences . "" xxmaj that said i have posted a full review here on our xxunk at xxunk : / / xxunk / 2014 / 12 / xxunk / our - experience - at - marvel - experience / for all to see complete with pictures if you would like more details about my comments below \n\n xxmaj xxunk : \n xxmaj it should be xxunk that i xxunk from reading any reviews or watching any news xxunk about this event prior to our visit so i kept my own expectations as high as they should have been for the cost of the tickets . xxmaj also , while some may say it is not fair to compare something like this to "" disneyland "" , i disagree",2,1,0.23139896988868713,1.7064099311828613
1,"xxbos i always xxunk at the xxunk of going to a restaurant that xxunk in nachos . i would hear xxunk of my friend who would xxunk xxmaj nacho xxmaj daddy on his side of town for hours on end . i was so close to going the one xxmaj downtown , however i was xxunk home xxunk by my significant other . \n\n xxmaj this time . xxmaj this time , it was xxup xxunk ! xxmaj my friends and i counted the days to opening when we first saw it was coming to our side of town . xxmaj the the time came . xxmaj we sat , we ate , we got full . xxmaj we left happy . \n\n xxmaj this place was awesome . xxmaj great ambiance . xxmaj awesome xxmaj servers . xxmaj insane menu ! xxmaj the prices may be a tiny bit",5,1,0.2272447943687439,1.7035293579101562


When I tried to improve the model with freezing, the accuray actually improved already in the first epoach, compared to the previous approach!

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.915854,3.954830,0.268000,52.186817,00:27
